In [1]:
import requests, re
import pandas as pd

from bs4 import BeautifulSoup
from typing import Dict
from time import sleep
from tqdm import tqdm_notebook

C:\Users\User\AppData\Local\Temp\ipykernel_12144\1251454757.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
start_page = requests.get("https://rus-ege.sdamgia.ru/archive", headers={'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"})

In [3]:
BASE_ADDR = "https://rus-ege.sdamgia.ru"

In [4]:
start_page

<Response [200]>

In [5]:
soup = BeautifulSoup(start_page.text)

In [6]:
variant_links = []

In [7]:
for span in soup.findAll("span", attrs={"class": "our_test pinkmark"}):
    variant_links.append(span.findChild("a").attrs["href"])

In [8]:
question = BeautifulSoup(requests.get(variant_links[0]).text).findChild("div", attrs={"class": "prob_num"}, string="22").findNext().find("div", attrs={"class": "pbody"})

In [9]:
BeautifulSoup(requests.get(variant_links[0]).text).findChild("div", attrs={"class": "prob_num"}, string="22").findNext().find("div", attrs={"class": "probtext"}).text.replace("\xad", "").replace("\xa0", " ").replace("\u202f", " ")

'(1)На задворках нашего села стояло на сваях длинное помещение из досок. (2)Я первый раз в жизни здесь услышал музыку  — скрипку. (3)На ней играл Вася-поляк. (4)О чём мне рассказывала музыка? (5)О чём-то очень большом. (6)На что же это жаловалась она, на кого гневалась? (7)Тревожно и горько мне. (8)Хочется заплакать, оттого что мне жалко самого себя, жалко тех, что спят непробудным сном на кладбище! (9)Вася, не переставая играть, говорил: «(10)Эту музыку написал человек, которого лишили самого дорогого. (11)Если у человека нет матери, нет отца, но есть родина, он ещё не сирота. (12)Всё проходит: любовь, сожаление о ней, горечь утрат, даже боль от ран,  — но никогда не проходит и не гаснет тоска по родине. (13)Эту музыку написал мой земляк Огинский. (14)Написал на границе, прощаясь с родиной. (15)Он посылал ей последний привет. (16)Давно уже нет композитора на свете, но боль его, тоска его, любовь к родной земле, которую никто не может отнять, жива до сих пор». (17)«Спасибо вам, дяденьк

In [10]:
BeautifulSoup(requests.get(variant_links[0]).text).findChild("div", attrs={"class": "prob_num"}, string="22").findNext().find("a").attrs["href"]

'/problem?id=294'

In [11]:
task_link = BeautifulSoup(requests.get(variant_links[0]).text).findChild("div", attrs={"class": "prob_num"}, string="22").findNext().find("a").attrs["href"]
task_link = BASE_ADDR + task_link
print(task_link)

https://rus-ege.sdamgia.ru/problem?id=294


In [12]:
task_soup = BeautifulSoup(requests.get(task_link).text)
qdata = task_soup.find("div", attrs={"class": "pbody"})
qlines = re.sub("<.*?>", "\n", str(qdata)).replace("\xad", "").replace("\xa0", "").replace("\u202f", "").strip().split('\n')

reading_text = task_soup.find("div", attrs={"class": "probtext"}).text.replace("\xad", "").replace("\xa0", " ").replace("\u202f", " ")
answer_text = task_soup.find("div", attrs={"class": "answer"}).text
datum = {"qlines": qlines, "reading_text": reading_text, "answer_text": answer_text}

In [13]:
def get_data_from_variant(variant_link: str, task_id: int) -> Dict:
    task_link = BeautifulSoup(requests.get(variant_link).text).findChild("div", attrs={"class": "prob_num"}, string=str(task_id)).findNext().find("a").attrs["href"]
    task_link = BASE_ADDR + task_link

    task_soup = BeautifulSoup(requests.get(task_link).text)
    qdata = task_soup.find("div", attrs={"class": "pbody"})
    qlines = re.sub("<.*?>", "\n", str(qdata)).replace("\xad", "").replace("\xa0", "").replace("\u202f", "").strip().split('\n')

    reading_text = task_soup.find("div", attrs={"class": "probtext"}).text.replace("\xad", "").replace("\xa0", " ").replace("\u202f", " ")
    answer_text = task_soup.find("div", attrs={"class": "answer"}).text
    datum = {"qlines": qlines, "reading_text": reading_text, "answer_text": answer_text}
    
    return datum

In [14]:
parsed_data = []

for variant_link in tqdm_notebook(variant_links, total=len(variant_links)):
    parsed_data.append(get_data_from_variant(variant_link, 22))
    sleep(0.5)

C:\Users\User\AppData\Local\Temp\ipykernel_12996\2694031947.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for variant_link in tqdm_notebook(variant_links, total=len(variant_links)):


  0%|          | 0/180 [00:00<?, ?it/s]

In [15]:
parsed_data_ = [
    {
        "question": " ".join(item["qlines"][:2]),
        "variants": [
            line for line in item["qlines"] if line.startswith("1)") or line.startswith("2)") or line.startswith("3)") or line.startswith("4)") or line.startswith("5)")
        ],
        "reading_text": item["reading_text"],
        "right_answers": [int(i) for i in list(item["answer_text"].split()[1].split("|")[0])]
    } for item in parsed_data
]

In [19]:
parsed_data_ = pd.DataFrame(parsed_data_)

In [21]:
parsed_data_.question.value_counts()

question
Какие из высказываний соответствуют содержанию текста? Укажите номера ответов. Цифры укажите в порядке возрастания.                                                                                                     77
Какие из высказываний соответствуют содержанию текста? Укажите номера ответов.                                                                                                                                          51
Какие из высказываний соответствуют содержанию текста? Укажите номера ответов в возрастающем порядке.                                                                                                                   22
Какие из высказываний  соответствуют                                                                                                                                                                                     6
Какие из высказываний  не соответствуют                                                                            

In [23]:
parsed_data_ = parsed_data_[parsed_data_.question.str.startswith("Какие из высказываний")]

In [24]:
parsed_data_.question.value_counts()

question
Какие из высказываний соответствуют содержанию текста? Укажите номера ответов. Цифры укажите в порядке возрастания.    77
Какие из высказываний соответствуют содержанию текста? Укажите номера ответов.                                         51
Какие из высказываний соответствуют содержанию текста? Укажите номера ответов в возрастающем порядке.                  22
Какие из высказываний  соответствуют                                                                                    6
Какие из высказываний  не соответствуют                                                                                 6
Какие из высказываний не соответствуют содержанию текста? Укажите номера ответов.                                       3
Какие из высказываний  противоречат                                                                                     3
Name: count, dtype: int64

In [25]:
parsed_data_

,question,variants,reading_text,right_answers
0,Какие из высказываний соответствуют содержанию...,"[1)Рассказчик был сиротой., 2)В детстве эта ме...",(1)На задворках нашего села стояло на сваях дл...,"[1, 2, 3]"
1,Какие из высказываний соответствуют содержанию...,"[1)Собеседник рассказчика, Сокольский, сомнева...","(1)\tМы ехали берегом Лены на юг, а зима догон...","[1, 2, 3]"
2,Какие из высказываний соответствуют содержанию...,[1)Родион встретил девочку перед наступлением....,"(1)Воспалённое состояние Поли, а главное, её с...","[2, 3]"
3,Какие из высказываний соответствуют содержанию...,[1)Ребята сразу невзлюбили своего классного ру...,"(1)Все мы любили «классного», хотя нельзя сказ...","[2, 3, 5]"
4,Какие из высказываний соответствуют содержанию...,"[1)Убежище, в котором укрывалась рассказчица, ...",(1)В суровые военные годы во время бомбёжки мо...,"[1, 2, 3]"
...,...,...,...,...
175,Какие из высказываний соответствуют,[1)Уолтер Грипп когда-то считал Тимоти Саллива...,(1)На седьмой странице был некролог: «(2)Тимот...,"[2, 4]"
176,Какие из высказываний соответствуют содержанию...,[1)Студенты давно собирались сбить замок с Миш...,"(1)Шла война, на которую мы, шестнадцатилетние...","[2, 4, 5]"
177,Какие из высказываний соответствуют содержанию...,"[1) Каждый раз, когда Архимед садился в ванну,...",(1)Однажды Архимед сел в ванну и вдруг почувст...,"[3, 4]"
178,Какие из высказываний соответствуют содержанию...,"[1)По мнению рассказчика, англичане пренебрежи...",(1)В маленьком городишке Стратфорде на маленьк...,"[4, 5]"


In [27]:
parsed_data_.to_csv("parsed_data.csv", sep=";")